In [1]:
!wget https://www.dropbox.com/s/41tegdrfqznjlzc/news.csv

--2020-07-04 00:06:59--  https://www.dropbox.com/s/41tegdrfqznjlzc/news.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/41tegdrfqznjlzc/news.csv [following]
--2020-07-04 00:06:59--  https://www.dropbox.com/s/raw/41tegdrfqznjlzc/news.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5ae2c13c498c187203d346450b.dl.dropboxusercontent.com/cd/0/inline/A61aoOMLM2VX8wnzrG2_s8qREOaoEdoOVlB9eD7hfhxksSNj9-DFkQW4vIjveDqEKWnmpT1XqlhP6CZn9zELcblpSlSpNR532kGjwA_l9D40pBu_WKcmhj_2lr1gmY6S2-I/file# [following]
--2020-07-04 00:07:00--  https://uc5ae2c13c498c187203d346450b.dl.dropboxusercontent.com/cd/0/inline/A61aoOMLM2VX8wnzrG2_s8qREOaoEdoOVlB9eD7hfhxksSNj9-DFkQW4vIjveDqEKWnmpT1XqlhP6CZn9zELcblpSlSpNR532kGjwA_l9D40pBu_WKcmhj_2lr1

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
import torch
import torch.nn as nn
import torchtext 
from torchtext.data import TabularDataset, Field, BucketIterator
from tqdm import tqdm
tqdm.pandas()

### Data Preprocessing
- Dataset Source: https://www.kaggle.com/nopdev/real-and-fake-news-dataset

In [4]:
data = pd.read_csv("news.csv")

In [5]:
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
# create length col
data['len'] = data['text'].progress_apply(lambda t : len(t))

100%|██████████| 6335/6335 [00:00<00:00, 414400.03it/s]


In [7]:
data.len.describe()

count      6335.000000
mean       4707.250355
std        5090.956446
min           1.000000
25%        1741.500000
50%        3642.000000
75%        6192.000000
max      115372.000000
Name: len, dtype: float64

In [8]:
#### Cleaning -> drop rows with empty text
data.drop(data[data.len < 10].index, inplace=True)

In [9]:
data.head()

,Unnamed: 0,title,text,label,len
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,7518
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,2646
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,2543
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,2660
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1840


In [10]:
data['label'] = (data['label'] == 'FAKE').astype('int')

In [11]:
data.head()

,Unnamed: 0,title,text,label,len
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",1,7518
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,1,2646
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,0,2543
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1,2660
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,0,1840


In [12]:
data = data[["text", "label"]]

In [13]:
train, test = train_test_split(data, test_size=0.1)

In [14]:
train, valid = train_test_split(train, test_size=0.1)

In [15]:
print(train.shape, test.shape, valid.shape)

(5102, 2) (630, 2) (567, 2)


In [16]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)
valid.to_csv("valid.csv", index=False)

#### TorchText Preprocessing

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [20]:
max_len = 5000

In [21]:
label_field = Field(sequential=False, use_vocab=False, dtype=torch.float, batch_first=True, is_target=True)
text_field = Field(tokenize="spacy", lower=True, fix_length=max_len, include_lengths=True, batch_first=True)

In [22]:
fields = [("text", text_field), ("label", label_field)]

In [23]:
train, test, valid = TabularDataset.splits(
    path="./", 
    train="train.csv", 
    validation="valid.csv", 
    test="test.csv",
    format="CSV", 
    fields=fields, 
    skip_header=True)

In [24]:
text_field.build_vocab(train,  min_freq=3)

In [26]:
batch_size = 128
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [29]:
train_loader = BucketIterator(
    dataset=train,
    batch_size=batch_size,
    sort_key=lambda x : len(x.text),
    sort=True,
    sort_within_batch=True,
    device=device,

)
test_laoder = BucketIterator(
    dataset=test,
    batch_size=batch_size,
    sort_key=lambda x : len(x.text),
    sort_within_batch=True,
    sort=True,
    device=device,
)
valid_loader = BucketIterator(
    dataset=valid,
    batch_size=batch_size,
    sort_key=lambda x : len(x.text),
    sort=True,
    sort_within_batch=True,
    device=device,
)

In [30]:
for (text, _), labels in train_loader:
    print(type(text), type(labels))
    print(text.shape, labels.shape)
    break

<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([128, 5000]) torch.Size([128])


In [31]:
len(train_loader)

40

### Model 

In [32]:
class LSTM(nn.Module):
    
    def __init__(self, embedding_dim=300, hidden_dim=128, num_classes = 1):
        super(LSTM, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(num_embeddings=len(text_field.vocab), embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            bidirectional=True
        )
        self.dropout = nn.Dropout(p=0.2)
        self.out = nn.Linear(in_features=2*hidden_dim, out_features=num_classes)
        
    def forward(self, text, text_len):
        
        embedded = self.embedding(text)
        packed_input = nn.utils.rnn.pack_padded_sequence(input=embedded, lengths=text_len, batch_first=True, enforce_sorted=False)
        packed_output, _ = self.lstm(packed_input)
        output, _ = nn.utils.rnn.pad_packed_sequence(sequence=packed_output, batch_first=True)
        
        out_forward = output[range(len(output)), text_len - 1, :self.hidden_dim]
        out_reverse = output[:, 0, self.hidden_dim:]
        out_reduced = torch.cat((out_forward, out_reverse), 1)
        text_fea = self.dropout(out_reduced)

        text_fea = self.out(text_fea)
        text_fea = torch.squeeze(text_fea, 1)
        text_out = torch.sigmoid(text_fea)

        return text_out
        

### Training

In [33]:
import torch.utils.tensorboard as tensorboard

In [34]:
def accuracy(y, y_hat):
    y_hat = [1 if x.item() > 0.5 else 0 for x in y_hat]
    correct = 0
    for i in range(len(y_hat)):
        if y_hat[i]==y[i].item():
            correct +=1
    acc = correct/len(y_hat)
    return acc

def eval(model, data_loader):
    with torch.no_grad():
        acc = []
        losses = []
        for step, ((text, text_len), labels) in enumerate(data_loader):
            text = text.to(device)
            labels = labels.to(device)
            text_len = text_len.to(device)
            outputs = model(text, text_len)
            losses.append(criterion(outputs, labels).item())
            acc.append(accuracy(labels, outputs.detach()))
        return sum(acc)/len(acc), sum(losses)/len(losses)

In [42]:
# model, loss, optimizer and tensorboard logger
model = LSTM().to(device)
criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
writer = tensorboard.SummaryWriter()

In [43]:
steps = 0
epochs = 20
epoch_progress = tqdm(total=epochs, desc="Epoch", position=0)

for epoch in range(epochs):
    epoch_train_loss = []
    epoch_train_acc = []
    
    step_progress = tqdm(total=len(train_loader), desc="Step", position=0)
    
    for step, ((text, text_len), labels) in enumerate(train_loader):
        
        text = text.to(device)
        labels = labels.to(device)
        text_len = text_len.to(device)
        
        outputs = model(text, text_len)
        
        optimizer.zero_grad()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        acc = accuracy(labels, outputs.detach())
        val_acc, val_loss = eval(model, valid_loader)

        if steps%20==0:
            print(f'Epoch {epoch} | Step {step} | Train_loss {loss.item():.4f} | Train_acc {acc:.4f} | Val_loss {val_loss:.4f} | Val_acc {val_acc:.4f}')

        writer.add_scalar("Step Training_loss", loss.item(), steps)
        writer.add_scalar("Step Training_acc", acc, steps)
        writer.add_scalar("Step Val_acc", val_acc, steps)
        writer.add_scalar("Step Val_loss", val_loss, steps)
        
        epoch_train_loss.append(loss.item())
        epoch_train_acc.append(acc)
        
        step_progress.update(1)
        steps +=1 
        
    avg_loss = sum(epoch_train_loss)/len(epoch_train_loss)
    avg_acc = sum(epoch_train_acc)/len(epoch_train_acc)
    val_acc, val_loss = eval(model, valid_iter)
    print(f"Epoch {epoch} | Train_loss {avg_loss:.4f} | Train_acc {avg_acc:.4f} | Val_loss {val_loss:.4f} | Val_acc {val_acc:.4f}")
    
    writer.add_scalar("Epoch Train_loss", loss.item(), steps)
    writer.add_scalar("Epoch Train_acc", acc, steps)
    writer.add_scalar("Epoch Val_acc", val_acc, steps)
    writer.add_scalar("Epoch Val_loss", val_loss, steps)
    
    epoch_progress.update(1)

Step:   2%|▎         | 1/40 [00:01<00:56,  1.46s/it]

Epoch 0 | Step 0 | Train_loss 0.2550 | Train_acc 0.2734 | Val_loss 0.2425 | Val_acc 0.6702


Step:  52%|█████▎    | 21/40 [00:31<00:30,  1.61s/it]

Epoch 0 | Step 20 | Train_loss 0.1763 | Train_acc 0.6797 | Val_loss 0.1788 | Val_acc 0.7095


Step:   0%|          | 0/40 [00:00<?, ?it/s]

...

Epoch 19 | Step 0 | Train_loss 0.0000 | Train_acc 1.0000 | Val_loss 0.0153 | Val_acc 0.9827


Step:  52%|█████▎    | 21/40 [00:30<00:29,  1.57s/it]

Epoch 19 | Step 20 | Train_loss 0.0000 | Train_acc 1.0000 | Val_loss 0.0154 | Val_acc 0.9811


Epoch: 100%|██████████| 20/20 [24:35<00:00, 72.78s/it]

Epoch 19 | Train_loss 0.0000 | Train_acc 1.0000 | Val_loss 0.0158 | Val_acc 0.9796


In [46]:
test_acc, test_loss = eval(model, test_laoder)
print(f"test_loss {test_loss} | test_acc {test_acc}")

test_loss 0.020353576634079218 | test_acc 0.9755397727272728


### Reference: https://towardsdatascience.com/lstm-text-classification-using-pytorch-2c6c657f8fc0